## Outils pour la manipulation d'images et librairies.


In [1]:
from PIL import Image
import numpy as np
import scipy as sp
import os
from math import log10, sqrt

def load(filename):
    toLoad= Image.open(filename)
    return np.asarray(toLoad)


def psnr(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

def dct2(a):
    return sp.fft.dct( sp.fft.dct( a, axis=0, norm='ortho' ), axis=1, norm='ortho' )

def idct2(a):
    return sp.fft.idct( sp.fft.idct( a, axis=0 , norm='ortho'), axis=1 , norm='ortho')

def save(a, path):
    Image.fromarray(a).save(path)
    



## Normalisation de l'image (YCbCr et padding)

#### RGB/YCbCr

In [2]:
def to_Y_Cb_Cr(filename): 
    '''transforme l'img en RGB en YCbCr'''
    image = load(filename)
    Y = 0.299*image[:, :, 0] + 0.587*image[:, :, 1] + 0.114*image[:, :, 2]
    Cb = -0.1687*image[:, :, 0] - 0.3313*image[:, :, 1] + 0.5*image[:, :, 2] + 128
    Cr = 0.5*image[:, :, 0] - 0.4187*image[:, :, 1] - 0.0813*image[:, :, 2] +128
    return Y, Cb, Cr #pas des int -> perte de l'information sur l'img

In [212]:
def to_RGB(Y, Cb, Cr): 
    '''transforme l'img en YCbCr en RGB'''
    R = np.array(np.rint(Y + 1.402*(Cr-128)))
    G = np.array(np.rint(Y - 0.34414*(Cb-128) - 0.71414*(Cr-128)))
    B = np.array(np.rint(Y + 1.772*(Cb-128)))
    image = np.dstack((R,G, B))
    image[image < 0] = 0
    image[image > 255] = 255
    
    return image

#### Pad/Unpad

In [4]:
def padding(image):
    '''cree le padding pour avoir les dimenstions de l'images divisibles par 8'''
    im_shape = image.shape
    y = im_shape[0]//8
    y = 8 * (y+1)
    x = im_shape[1]//8
    x = 8 * (x+1)

    append1 = np.zeros((im_shape[0], x-im_shape[1]), dtype=int) #ajout de 0 "a droite de l'image"
    append2 = np.zeros((y-im_shape[0], x), dtype=int) #ajout de 0 "en bas de l'image"
    image = np.concatenate((image, append1), axis=1)
    image = np.concatenate((image, append2), axis=0) #concatenation avec l'image
    image = np.array(image)

    return image, im_shape

In [5]:
def unpad(image, im_shape):
    '''remene l'img a sa dimension initiale'''
    image = np.array(image[:im_shape[0], :im_shape[1]])
    
    return image

#### x2 / /2

In [6]:
def diviser_par_deux(C):
    '''divise par 2 la longeur des crominances'''
    im_shape = C.shape
    C1 = np.empty((im_shape[0], int(im_shape[1]/2)))
    for i in range(im_shape[0]):
        for j in range(int(im_shape[1]/2)):
            C1[i, j] = (C[i, j*2]+ C[i, j*2+1])/2
    return C1

In [7]:
def multiplier_par_deux(C):
    '''multiplie par 2 la longeur des crominances'''
    im_shape = C.shape
    C1 = np.empty((im_shape[0], im_shape[1]*2))
    for i in range(im_shape[0]):
        for j in range(im_shape[1]):
            C1[i, j*2] = C[i, j]
            C1[i, j*2+1] = C[i, j]
    return C1

## Découpage en blocs et compression

#### decoupage

In [10]:
def decouper(matrix):
    '''decoupe l'image en blocks 8*8'''
    l_matrix = []
    im_shape = matrix.shape
    for i in range(0, im_shape[0], 8):
        for j in range(0, im_shape[1], 8):
            l_matrix.append(matrix[i:i+8, j:j+8])
    return l_matrix

In [230]:
def assembler(blocks, im_shape):
    '''rassemble les blocks de 8*8 en une image
    im_shape : valeur de la dimension divisee par 8'''

    blocks1 = np.empty((im_shape[0], 8, (im_shape[1]*8)))

    for i in range(im_shape[0]):
        blocks1[i, :, :] = np.concatenate(blocks[((im_shape[1]*i)):(im_shape[1]*(i+1)), :, :], axis = 1) #assemble les blocks ligne par ligne
    blocks = np.concatenate(blocks1, axis = 0) #assemble les lignes

    return blocks

#### dct/idct

In [12]:
def appliquer_dct(l_matrix):
    for i in range(len(l_matrix)):
        l_matrix[i] = dct2(l_matrix[i]).astype('int')
    return l_matrix

In [13]:
def appliquer_idct(l_matrix):
    for i in range(l_matrix.shape[0]):
        l_matrix[i] = idct2(l_matrix[i])
    return l_matrix

#### seuil

In [14]:
def seuil(blocks, s):
    '''retourne des valeurs dont la valeur absolue est supperieur au seuil'''
    for b in blocks:
        b[abs(b) < s] = 0
    return blocks

#### RLE/UnRLE

In [15]:
def rle(blocks):
    '''remplace les 0 par leur nombre de repetitions consecutives'''
    blocks1 = []
    for b in blocks:
        b1 = []
        b = b.astype('str')
        repetition = 0
        for i in range(8):
            for j in range(8):

                if b[i, j] == '0':
                    repetition += 1
                else:
                    if repetition != 0:
                        b1.append(f'#{repetition}')
                        repetition = 0
                    b1.append(b[i, j])

        if repetition != 0:
            b1.append(f'#{repetition}')
        
        blocks1.append(b1)
    
    return blocks1




In [16]:
def unrle(matrix):
    '''remplace le mb de repetition de 0 par 0'''
    matrix = matrix.split(' ')
    blocks = []
    for i in range(len(matrix)):
        if matrix[i][0] == '#':
            for j in range(int(matrix[i][1:])):
                blocks.append(0)
        else:
            blocks.append(int(matrix[i]))
    blocks = np.array(blocks)
    blocks = np.reshape(blocks, (8,8))
    return blocks


### Bonus

#### modes facultatives

In [17]:
def mode3(blocks):
    '''sauvegardes les 8 valeurs les plus grandes'''
    for i in range(len(blocks)):
        limit =  np.sort(np.absolute(blocks[i].copy()), axis=None)[-8] 
        blocks[i][abs(blocks[i]) < limit] = 0 #les valeurs absolues sont prises pour calculer la valeur limite
    return blocks

In [18]:
Q = np.array([[16, 11, 10, 16, 24, 40, 51, 61],
     [12, 12, 14, 19, 26, 58, 60, 55],
     [14, 13, 16, 24, 40, 57, 69, 56],
     [14, 17, 22, 29, 51, 87, 80, 62],
     [18, 22, 37, 56, 68, 109, 103, 77],
     [24, 35, 55, 64, 81, 104, 113, 92],
     [49, 64, 78, 87, 103, 121, 120, 101],
     [72, 92, 95, 98, 112, 100, 103, 99]])

In [232]:
def mode4_compress(blocks):
    '''divise les valeurs des blocks par Q'''
    global Q
    for i in range(len(blocks)):
        blocks[i] = (blocks[i]/Q).astype('int')
    return blocks

In [233]:
def mode4_decompress(blocks):
    '''multiplie les valeurs des blocks par Q'''
    global Q
    for i in range(len(blocks)):
        blocks[i] = (blocks[i]*Q)
    return blocks    

#### encodage en Zig-Zag

In [227]:
def Zig_zag_rle(blocks):

    blocks1 = []
    for i in range(len(blocks)):
        b1 = []
        for j in range(0, 64):
            if j < 8:
                p_carre = [w[:j] for w in blocks[i][:j]] #prend le carre de taille j*j en haut a gauche du carre 8*8
            else:
                p_carre = [w[j-9:] for w in blocks[i][j-9:]] #prend le carre de taille j*j en en bas a droite du carre 8*8
            diagonale = [p_carre[w][len(p_carre)-w-1] for w in range(len(p_carre))] #prend la diagonale du carre j*j
            if len(diagonale) % 2:
                diagonale.reverse() 
            b1 += diagonale 

        b2 = [] #rle
        count_0 = 0
        for j in range(0, 65):
                if b1[j] == 0:
                    count_0 += 1
                else:
                    if count_0 != 0:
                        b2.append(f'#{count_0}')
                        count_0 = 0
                    b2.append(str(b1[j]))
        if count_0 != 0:
            b2.append(f'#{count_0}')
        blocks1.append(b2)
    return blocks1

In [128]:
def Zig_zag_unrle(matrix):
    
    matrix = matrix.split(' ') #unrle
    blocks = []
    for i in range(len(matrix)):
        if matrix[i][0] == '#':
            for j in range(int(matrix[i][1:])):
                blocks.append(0)
        else:
            blocks.append(int(matrix[i]))
    
    blocks1_1 = np.array([[0]*8 for _ in range(8)])
    blocks1_2 = np.array([[0]*8 for _ in range(8)])
    to = 8
    for b in [blocks1_1, blocks1_2]: #fait 2 demi-matrices
        total = 0
        for i in range(0,to):
            for j in range(i+1):
                if i%2==0:
                    b[i-j, j] = blocks[total] #reconstuit la matrice 8*8 en diagonale
                    total += 1
                else:
                    b[j, i-j] = blocks[total]
                    total += 1
        to -= 1
        blocks = blocks[total:]
        blocks.reverse() 
    
    result = np.maximum(blocks1_1, blocks1_2[::-1,::-1]) #renverse la 2e demi-matrice et unifie les 2
      
    return result

## Compression

#### rassembler le tout

In [142]:
def compress(img_path, mode, nb_de_seuil):
    '''applique la compression'''
    y, cb, cr = to_Y_Cb_Cr(img_path)
    img = [y, cb, cr]

    for i in range(3):

        if mode >= 2 and i != 0:
            img[i] = diviser_par_deux(img[i])

        if i == 0:
            img[i], initial_shape = padding(img[i])
        else:
            img[i], _ = padding(img[i])


        img[i] = appliquer_dct(decouper(img[i]))
        
        if mode == 1 or mode == 2:
            img[i] = seuil(img[i], nb_de_seuil)
            
        elif mode == 3:
            img[i] = mode3(img[i])
        elif mode == 4:
            img[i] = mode4_compress(img[i])
        

    return img[0], img[1], img[2], initial_shape

#### Écriture dans un fichier

In [24]:
def block_en_ligne(block):
    '''transforme int blocks 8*8 en str array 1*64'''
    block = np.array(block)
    block = block.astype('str')   
    block = np.reshape(block, (block.shape[0], 64))
    return block.tolist()

In [244]:
def assertions(mode, use_rle, nb_de_seuil):
    assert type(mode) == int and mode >= 0 and mode < 5, \
        'Veillez de choisir un mode int compris entre 0 et 4'
    assert type(use_rle) == str and use_rle in ['NORLE', 'RLE', 'ZIGZAG_RLE'], \
        "Veillez de choisir use_rle in ['NORLE', 'RLE', 'ZIGZAG_RLE']"
    assert type(nb_de_seuil) == int and nb_de_seuil >= 0, \
        "Veillez de choisir nombre de seuil int positif"

In [236]:
def write_file(img_path, mode = 2, use_rle = 'RLE', nb_de_seuil = 4):
    '''ecrit le fichier txt avec l'img compresse'''
    assertions(mode, use_rle, nb_de_seuil)
    y, cb, cr, initial_shape = compress(img_path, mode, nb_de_seuil)
    #y, cb, cr = np.around(y), np.around(cb, 1), np.around(cr, 1)
    f = open(img_path[:-4]+'_compressed.txt', 'w')

    f.write('SJPG\n')
    f.write(f'{initial_shape[0]} {initial_shape[1]}\n') 
    f.write(f'mode {mode}\n')
    f.write(f'{use_rle}\n')

    img = [y, cb, cr]
    for i in range(3):
        if use_rle == 'NORLE':
            img[i] = block_en_ligne(img[i])
        elif use_rle == 'RLE':
            img[i] = rle(img[i])
        elif use_rle == 'ZIGZAG_RLE':
            img[i] = Zig_zag_rle(img[i])

        for j in img[i]:
            f.write(' '.join(j))
            f.write('\n')
    
    f.close()
 

#### executer compression

In [26]:
'''write(img_name, mode_nb, use_rle, nb_de_seuil)
params: mode: int 0, 1, 2, 3, 4
        use_rle: str NORLE, RLE, ZIGZAG_RLE
        nb_de_seuil: int positif
'''
write_file('test.png', mode = 4, use_rle = 'RLE', nb_de_seuil = 4)

## Décompression

In [118]:
def str_to_array(padding, im_shape1, lines, use_RLE):
    '''transforme la ligne en un block 8*8 et execute le unrle'''
    for i in range(2):
        im_shape1[i] = im_shape1[i] //8
        if im_shape1[i] %8 != 0:
            im_shape1[i]  += 1
    
    blocks = np.empty((int(im_shape1[0]*im_shape1[1]), 8, 8))
    for i in range(im_shape1[0]*im_shape1[1]):
        if use_RLE == 'RLE':
            blocks[i, :, :] = unrle(lines[i+padding].strip())
        elif use_RLE == 'ZIGZAG_RLE':
            blocks[i, :, :] = Zig_zag_unrle(lines[i+padding].strip())
        else:
            matrix = lines[i+padding].strip().split(' ')
            matrix = np.array(matrix)
            matrix = np.reshape(matrix, (8,8))
                
            blocks[i, :, :] = matrix
    
    return blocks, i+padding+1

In [198]:
def decompresser(path):
    '''decompresse l'image'''
    f = open(path, 'r')
    lines=f.readlines()
    f.close()

    im_shape = lines[1].strip().split(' ')
    im_shape[0] = int(im_shape[0])
    im_shape[1] = int(im_shape[1])
    
    im_shape_initial = im_shape.copy()

    mode = int(lines[2][5])
    use_rle = lines[3].strip()
    
    img = [0]*3

    padding = 4 #ligne ou commence le block Y
    for i in range(3):
        if mode >= 2:
            if i == 1:
                im_shape[1] = int(im_shape[1]/2)


        im_shape1 = im_shape.copy()
        img[i], padding = str_to_array(padding, im_shape1, lines, use_rle)

        if mode == 4:
            img[i] = mode4_decompress(img[i])

        img[i] = appliquer_idct(img[i])
        img[i] = assembler(img[i], im_shape1)
    
        if mode >= 2:
            if i != 0:
                img[i] = multiplier_par_deux(img[i])

        img[i] = unpad(img[i], im_shape_initial)
        
    img = to_RGB(img[0], img[1], img[2])
    save(img.astype('uint8'), f'{path[:-16]}_decompresse.png')

In [250]:
write_file('test.png', mode = 3, use_rle = 'RLE', nb_de_seuil = 30)

In [ ]:
decompresser('test_compressed.txt')

## Tests 

In [252]:
test = load("test.png")
test1 = load("test_decompresse.png")
print(psnr(test, test1))

39.89633481399582
